In [8]:
# Load modules
%pylab inline
from netCDF4 import Dataset # to work with NetCDF files
import numpy as np
import matplotlib.pyplot as plt # to generate plots
from mpl_toolkits.basemap import Basemap # plot on map projections
from os.path import expanduser
home = expanduser("~") # Get users home directory
import os # operating system interface
import scipy as sp
import xarray as xr
from xarray.ufuncs import square, sqrt
import scipy.interpolate
import scipy.ndimage

Populating the interactive namespace from numpy and matplotlib


In [9]:
#pip install git+https://github.com/OceanParcels/parcels.git@master --user

In [10]:
%matplotlib inline
from parcels import FieldSet, ParticleSet, Variable, JITParticle, AdvectionRK4, plotTrajectoriesFile, ParticleFile
import numpy as np
import math
from datetime import timedelta

TypeError: mkdir() got an unexpected keyword argument 'exist_ok'

In [ ]:
filenames = "/g/data3/hh5/tmp/pas561/gfdl_nyf_1080_PI_Ant_20Eto140E/5yrs_5day/IC200/output201/ocean__201_00*.nc"
variables = {'U': 'u',
             'V': 'v',
             'W': 'wt',
            'temp': 'temp'}
dimensions = {'lat': 'geolat_t',
              'lon': 'geolon_t',
              'depth':'st_ocean',
              'time': 'time'}
fieldset = FieldSet.from_netcdf(filenames, variables, dimensions,allow_time_extrapolation=True)
fieldset.add_periodic_halo(zonal=True,meridional=False,halosize=2)

In [ ]:
print fieldset.U.__dict__

In [ ]:
class tempparticle(JITParticle):
    temp=Variable('temp',initial=fieldset.temp)

In [ ]:
lons,lats,depths=np.meshgrid(np.arange(-10,10,2),np.arange(-65,-60,2),np.arange(10,5000,500))
print lons.shape
#time=-1 to go backwards
pset=ParticleSet(fieldset,tempparticle,lons,lats,depth=depths,time=fieldset.U.grid.time_full[0])

In [ ]:
pset.show()

In [ ]:
print fieldset.U.grid.lon

In [ ]:
def periodicbc(particle, fieldset, time, dt):
    if particle.lon >= 80:
        particle.lon -= 360
    if particle.lon <= -280:
        particle.lon += 360
        
def sampletemp(particle, fieldset, time, dt):
    particle.temp = fieldset.temp[time,particle.lon,particle.lat,particle.depth]

In [ ]:
pfile=pset.ParticleFile('myfirstparcels',outputdt=86400)

In [ ]:
pset.execute(pset.Kernel(periodicbc)+AdvectionRK4+sampletemp,dt=3600,runtime=86400*365,output_file=pfile)

In [ ]:
plotTrajectoriesFile('myfirstparcels.nc',mode='3d')